In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.distributions.empirical_distribution import ECDF
from functools import partial

/Users/francis/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data Handling
This notebook slices the data into training data and testing data

## Data
One single dataset from Bloomberg. 
- Daily closing price of BTC future and Daily Closing value of brr. 
- Data is merged by Bloomberg using the compare function


In [2]:
future_brr = pd.read_csv('../data/cleaned_data/btc_future_brr_new.csv')

In [29]:
future_brr

,Unnamed: 0,Date,btc future last px,brr last px,log return future,log return btc
0,1,2021-05-20,40320.0,41861.09,0.024606,0.158251
1,2,2021-05-19,39340.0,35734.10,-0.092441,-0.187849
2,3,2021-05-18,43150.0,43118.61,-0.020870,-0.028939
3,4,2021-05-17,44060.0,44384.66,-0.130263,-0.134395
4,8,2021-05-14,50190.0,50769.13,0.037039,0.003599
...,...,...,...,...,...,...
660,1076,2018-10-08,6630.0,6612.76,0.015198,0.010798
661,1080,2018-10-05,6530.0,6541.74,-0.003821,0.000035
662,1081,2018-10-04,6555.0,6541.51,0.020031,0.011635
663,1082,2018-10-03,6425.0,6465.84,-0.013143,-0.010170


In [30]:
future_brr = future_brr.iloc[:, 1:]

## Slicing data

In [31]:
train_size = 300
test_size = 5
step_size = 5

In [38]:
len(future_brr)

665

In [39]:
data_name = 'future_brr_new'

In [41]:
# n = int(np.floor((len(future_brr)-train_size)/step_size))
n = int(np.floor((len(future_brr) - train_size - test_size)/step_size))
train_set = []
test_set = []

for i in range(n):
    data = future_brr.iloc[(i*step_size):((i)*step_size)+train_size+test_size]
    test = data.iloc[:test_size,]
    train = data.iloc[test_size:,]
    train_set.append(train)
    test_set.append(test)

for i in range(len(train_set)):
    train_set[i].to_csv("../processed_data/%s/train/" %data_name +str(i)+".csv")
    test_set[i].to_csv("../processed_data/%s/test/" %data_name +str(i)+".csv")

In [34]:
test_set[0].head()

,Date,btc future last px,brr last px,log return future,log return btc
0,2021-05-20,40320.0,41861.09,0.024606,0.158251
1,2021-05-19,39340.0,35734.10,-0.092441,-0.187849
2,2021-05-18,43150.0,43118.61,-0.020870,-0.028939
3,2021-05-17,44060.0,44384.66,-0.130263,-0.134395
4,2021-05-14,50190.0,50769.13,0.037039,0.003599


In [35]:
train_set[0].head()

,Date,btc future last px,brr last px,log return future,log return btc
5,2021-05-13,48365.0,50586.73,-0.118415,-0.113057
6,2021-05-12,54445.0,56641.72,-0.042433,0.015200
7,2021-05-11,56805.0,55787.27,0.019554,-0.028760
8,2021-05-10,55705.0,57415.03,-0.036227,0.005824
9,2021-05-07,57760.0,57081.64,0.027736,-0.003132


In [36]:
train_set[0].tail()

,Date,btc future last px,brr last px,log return future,log return btc
300,2020-03-13,5330.0,5227.33,-0.120074,-0.152288
301,2020-03-12,6010.0,6087.20,-0.267726,-0.247034
302,2020-03-11,7855.0,7792.97,-0.017666,-0.018170
303,2020-03-10,7995.0,7935.86,0.018303,0.030342
304,2020-03-09,7850.0,7698.69,-0.154878,-0.164215


In [37]:
test_set[1].head()

,Date,btc future last px,brr last px,log return future,log return btc
5,2021-05-13,48365.0,50586.73,-0.118415,-0.113057
6,2021-05-12,54445.0,56641.72,-0.042433,0.015200
7,2021-05-11,56805.0,55787.27,0.019554,-0.028760
8,2021-05-10,55705.0,57415.03,-0.036227,0.005824
9,2021-05-07,57760.0,57081.64,0.027736,-0.003132
